In [1]:
import pandas as pd
import numpy as np
import json
import os

In [2]:
def dfLoaderJson(dataSrc) :
    dfs = []
    dataFiles = os.listdir(dataSrc)

    print(f'reading {len(dataFiles)} from {dataSrc}')

    for f in dataFiles:
        dfs.append(pd.read_json(f'{dataSrc}/{f}'))
    
    all_dfs = pd.concat(dfs)
    print(all_dfs.shape)

    return all_dfs


In [3]:
def cleanRankings(rankings_df):
    rankings_exploded = rankings_df.explode('polls')
    rankings_exploded['pollName'] = rankings_exploded['polls'].apply(lambda x: x['poll'] if pd.notna(x) else None)
    rankings_exploded['ranks'] = rankings_exploded['polls'].apply(lambda x: x['ranks'] if pd.notna(x) else None)

    rankings_flat = rankings_exploded.explode('ranks')
    rankings_flat['school'] = rankings_flat['ranks'].apply(lambda x:x['school'] if pd.notna(x) else None)
    rankings_flat['rank'] = rankings_flat['ranks'].apply(lambda x:x['rank'] if pd.notna(x) else None)
    rankings_flat['teamId'] = rankings_flat['ranks'].apply(lambda x:x['teamId'] if pd.notna(x) else None)
    rankings_flat[['season', 'seasonType', 'week', 'pollName', 'teamId','school','rank']].copy()

    #pivot on poll type
    rankings_pivot = rankings_flat.pivot_table(
        index=['season', 'week', 'teamId', 'school'],
        columns='pollName',
        values='rank',
        aggfunc='first'
    ).reset_index()

    rankings_pivot.columns = ['_'.join(col.split()).lower() for col in rankings_pivot.columns]

    return rankings_pivot

    

In [34]:
def mergeGamesWithRankings(games_df, rankings_df):
    #home
    gamesRankings_df = games_df.merge( 
        rankings_df, 
        left_on=['season', 'week', 'homeId'], 
        right_on=['season', 'week', 'teamid'], 
        how='left'
    ).rename( columns={
            'ap_top_25': 'home_ap_rank',
            'coaches_poll': 'home_coaches_rank',
            'playoff_committee_rankings': 'home_committee_rank'
        })


    #away
    gamesRankings_df = gamesRankings_df.merge( 
            rankings_df, 
            left_on=['season', 'week', 'awayId'], 
            right_on=['season', 'week', 'teamid'], 
            how='left'
        ).rename( columns={
            'ap_top_25': 'away_ap_rank',
            'coaches_poll': 'away_coaches_rank',
            'playoff_committee_rankings': 'away_committee_rank'
        })

    print(gamesRankings_df.shape)
    print(gamesRankings_df.columns)

    return gamesRankings_df

In [81]:
def calcRankWinPercent(df, awayPollRankCol, homePollRankCol, onlyBothRanked = True):
    if onlyBothRanked:
        comp_df = df[(df[awayPollRankCol] > 0) & (df[homePollRankCol] > 0)].copy()
    else:
        comp_df= df[(df[awayPollRankCol] > 0) | (df[homePollRankCol] > 0)].copy()
        comp_df[awayPollRankCol] = comp_df[awayPollRankCol].fillna(999)
        comp_df[homePollRankCol] = comp_df[homePollRankCol].fillna(999)
        
    print(comp_df.shape)
    higher_ranked_won_df = comp_df[
    ((comp_df[homePollRankCol] < comp_df[awayPollRankCol] ) 
     &(comp_df['homePoints'] > comp_df['awayPoints'])) 
     |
    ((comp_df[awayPollRankCol] < comp_df[homePollRankCol] ) 
     & (comp_df['awayPoints'] > comp_df['homePoints'])) ]
    
    wins = higher_ranked_won_df.shape[0]
    total = comp_df.shape[0]
    print(f'{wins}/{total}->{wins/total:.4f}')

    if onlyBothRanked:
        scope = 'both'
    else:
        scope = 'either'
    res = {
        'Poll': awayPollRankCol.replace('away_', ''),
        'Scope': f'{scope} ranked',
        'Wins': wins, 
        'Total': total, 
        'Accuracy' : wins/total
    }

    return res, higher_ranked_won_df

In [4]:
games_path = '../data/games'
games_df = dfLoaderJson(games_path)

games_df.head(5)

reading 26 from ../data/games
(45731, 33)


,id,season,week,seasonType,startDate,startTimeTBD,completed,neutralSite,conferenceGame,attendance,...,awayClassification,awayConference,awayPoints,awayLineScores,awayPostgameWinProbability,awayPregameElo,awayPostgameElo,excitementIndex,highlights,notes
0,63068,2000,1,regular,2000-08-26T00:00:00.000Z,False,True,False,False,NaN,...,fbs,Mountain West,3.0,[],NaN,1603.0,1572.0,NaN,,NaN
1,63070,2000,1,regular,2000-08-26T00:00:00.000Z,False,True,False,False,NaN,...,None,None,10.0,[],NaN,NaN,NaN,NaN,,NaN
2,63071,2000,1,regular,2000-08-26T00:00:00.000Z,False,True,False,False,NaN,...,fbs,Mountain West,3.0,[],NaN,1324.0,1289.0,NaN,,NaN
3,63069,2000,1,regular,2000-08-26T00:00:00.000Z,False,True,False,False,NaN,...,fbs,Big Ten,7.0,[],NaN,1344.0,1349.0,NaN,,NaN
4,63072,2000,1,regular,2000-08-27T00:00:00.000Z,False,True,False,False,NaN,...,fbs,Big Ten,5.0,[],NaN,1789.0,1712.0,NaN,,NaN


In [5]:
game_cols = ['week', 'season', 'homeId', 'homeTeam', 'homeConference', 'awayId', 'awayTeam', 'awayConference', 'homePoints','awayPoints']


games_df[game_cols].head(5)

,week,season,homeId,homeTeam,homeConference,awayId,awayTeam,awayConference,homePoints,awayPoints
0,1,2000,52,Florida State,ACC,252,BYU,Mountain West,29.0,3.0
1,1,2000,2348,Louisiana Tech,FBS Independents,2400,Mississippi Valley State,None,63.0,10.0
2,1,2000,2641,Texas Tech,Big 12,167,New Mexico,Mountain West,24.0,3.0
3,1,2000,2306,Kansas State,Big 12,2294,Iowa,Big Ten,27.0,7.0
4,1,2000,30,USC,Pac-10,213,Penn State,Big Ten,29.0,5.0


In [6]:
rankings_path = '../data/rankings'
rankings_df = dfLoaderJson(rankings_path)

rankings_df.head(5)

reading 26 from ../data/rankings
(414, 4)


,season,seasonType,week,polls
0,2000,regular,1,"[{'poll': 'AP Top 25', 'ranks': [{'rank': 17, ..."
1,2000,regular,2,"[{'poll': 'AP Top 25', 'ranks': [{'rank': 17, ..."
2,2000,regular,3,"[{'poll': 'AP Top 25', 'ranks': [{'rank': 17, ..."
3,2000,regular,4,"[{'poll': 'AP Top 25', 'ranks': [{'rank': 16, ..."
4,2000,regular,5,"[{'poll': 'AP Top 25', 'ranks': [{'rank': 11, ..."


In [7]:
sample_rankings_df = rankings_df[:10]

In [8]:
cleanRankings_df = cleanRankings(rankings_df)
cleanRankings_df[['season','week', 'teamid', 'school', 'ap_top_25', 'coaches_poll', 'playoff_committee_rankings']]

,season,week,teamid,school,ap_top_25,coaches_poll,playoff_committee_rankings
0,2000,1,2,Auburn,18.0,NaN,NaN
1,2000,1,30,USC,15.0,NaN,NaN
2,2000,1,36,Colorado State,14.0,NaN,NaN
3,2000,1,38,Colorado,24.0,NaN,NaN
4,2000,1,52,Florida State,2.0,NaN,NaN
...,...,...,...,...,...,...,...
18999,2025,12,2964,Central College,NaN,NaN,NaN
19000,2025,12,3071,North Central College,NaN,NaN,NaN
19001,2025,12,3112,Christopher Newport,NaN,NaN,NaN
19002,2025,12,110242,West Florida,NaN,NaN,NaN


In [9]:
week1_2024 = cleanRankings_df[
    (cleanRankings_df['season'] == 2024) & 
    (cleanRankings_df['week'] == 12)
]

week1_2024[['school', 'ap_top_25', 'fcs_coaches_poll', 'playoff_committee_rankings' ]]

,school,ap_top_25,fcs_coaches_poll,playoff_committee_rankings
17506,Colorado,18.0,NaN,17.0
17507,Georgia,11.0,NaN,12.0
17508,Iowa State,NaN,NaN,NaN
17509,Boise State,13.0,NaN,13.0
17510,Idaho,NaN,10.0,NaN
...,...,...,...,...
17603,PennWest California,NaN,NaN,NaN
17604,Ouachita Baptist,NaN,NaN,NaN
17605,Incarnate Word,NaN,6.0,NaN
17606,North Central College,NaN,NaN,NaN


In [10]:
rankings_by_school = cleanRankings_df[
    (cleanRankings_df['season'] == 2024) 
].pivot_table(
        index=['school'], 
        columns=['season', 'week'], 
        values=['ap_top_25'],
        aggfunc='first' 
    ).reset_index()

In [11]:
rankings_by_school

school ap_top_25                                            \
season                         2024                                             
week                              1     2     3     4     5     6     7     8   
0                 Alabama      17.0   4.0   4.0   4.0   4.0   1.0   7.0   7.0   
1                 Arizona      21.0  20.0  20.0   NaN   NaN   NaN   NaN   NaN   
2           Arizona State       7.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3                    Army      21.0   NaN   NaN   NaN   NaN   NaN   NaN  23.0   
4                     BYU      13.0   NaN   NaN   NaN  22.0  17.0  14.0  13.0   
5             Boise State       8.0   NaN   NaN   NaN  25.0  21.0  17.0  15.0   
6          Boston College       NaN   NaN  24.0   NaN   NaN   NaN   NaN   NaN   
7                 Clemson      14.0  25.0  22.0  21.0  17.0  15.0  10.0  10.0   
8                Colorado      25.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
9           Florida State      10.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
10                Georgia       6.0   1.0   1.0   2.0   2.0   5.0   5.0   5.0   
11           Georgia Tech       NaN  23.0   NaN   NaN   NaN   NaN   NaN   NaN   
12               Illinois      16.0   NaN   NaN  24.0  19.0  24.0  23.0  22.0   
13                Indiana      10.0   NaN   NaN   NaN   NaN  23.0  18.0  16.0   
14                   Iowa      25.0  21.0   NaN   NaN   NaN   NaN   NaN   NaN   
15             Iowa State      15.0   NaN  21.0  20.0  18.0  16.0  11.0   9.0   
16                 Kansas      22.0  19.0   NaN   NaN   NaN   NaN   NaN   NaN   
17           Kansas State      18.0  17.0  14.0  13.0  23.0  20.0  18.0  17.0   
18                    LSU      13.0  18.0  16.0  16.0  14.0  13.0  13.0   8.0   
19             Louisville       NaN  22.0  19.0  19.0  15.0  22.0   NaN   NaN   
20                Memphis      24.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
21                  Miami      18.0  12.0  10.0   8.0   7.0   8.0   6.0   6.0   
22               Michigan       9.0  10.0  17.0  18.0  12.0  10.0  24.0  24.0   
23               Missouri      22.0   9.0   6.0   7.0  11.0   9.0  21.0  19.0   
24               NC State      24.0  24.0   NaN   NaN   NaN   NaN   NaN   NaN   
25                   Navy       NaN   NaN   NaN   NaN   NaN   NaN   NaN  25.0   
26               Nebraska       NaN   NaN  23.0  22.0   NaN   NaN   NaN   NaN   
27      Northern Illinois       NaN   NaN  25.0  23.0   NaN   NaN   NaN   NaN   
28             Notre Dame       2.0   5.0  18.0  17.0  16.0  14.0  11.0  12.0   
29             Ohio State       1.0   2.0   3.0   3.0   3.0   3.0   2.0   4.0   
30               Oklahoma      16.0  15.0  15.0  15.0  21.0  19.0  18.0   NaN   
31         Oklahoma State      17.0  16.0  13.0  14.0  20.0   NaN   NaN   NaN   
32               Ole Miss      11.0   6.0   5.0   5.0   6.0  12.0   9.0  18.0   
33                 Oregon       3.0   7.0   9.0   9.0   8.0   6.0   3.0   2.0   
34             Penn State       5.0   8.0   8.0  10.0   9.0   7.0   4.0   3.0   
35             Pittsburgh       NaN   NaN   NaN   NaN   NaN   NaN  22.0  20.0   
36                    SMU      12.0   NaN   NaN   NaN   NaN   NaN  25.0  21.0   
37         South Carolina      19.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
38               Syracuse      20.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
39              Tennessee       9.0  14.0   7.0   6.0   5.0   4.0   8.0  11.0   
40                  Texas       4.0   3.0   2.0   1.0   1.0   2.0   1.0   1.0   
41              Texas A&M      20.0   NaN   NaN  25.0  24.0  25.0  15.0  14.0   
42                 Tulane       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
43                   UNLV      23.0   NaN   NaN   NaN   NaN  25.0   NaN   NaN   
44                    USC      23.0  13.0  11.0  11.0  13.0  11.0   NaN   NaN   
45                   Utah      12.0  11.0  12.0  12.0  10.0  18.0  16.0   NaN   
46             Vanderbilt       NaN   NaN   NaN   

In [35]:
gamesRankings_df = None

In [36]:
gamesRankings_df = mergeGamesWithRankings(games_df, cleanRankings_df)


(45731, 51)
Index(['id', 'season', 'week', 'seasonType', 'startDate', 'startTimeTBD',
       'completed', 'neutralSite', 'conferenceGame', 'attendance', 'venueId',
       'venue', 'homeId', 'homeTeam', 'homeClassification', 'homeConference',
       'homePoints', 'homeLineScores', 'homePostgameWinProbability',
       'homePregameElo', 'homePostgameElo', 'awayId', 'awayTeam',
       'awayClassification', 'awayConference', 'awayPoints', 'awayLineScores',
       'awayPostgameWinProbability', 'awayPregameElo', 'awayPostgameElo',
       'excitementIndex', 'highlights', 'notes', 'teamid_x', 'school_x',
       'afca_division_ii_coaches_poll_x', 'afca_division_iii_coaches_poll_x',
       'home_ap_rank', 'bcs_standings_x', 'home_coaches_rank',
       'fcs_coaches_poll_x', 'home_committee_rank', 'teamid_y', 'school_y',
       'afca_division_ii_coaches_poll_y', 'afca_division_iii_coaches_poll_y',
       'away_ap_rank', 'bcs_standings_y', 'away_coaches_rank',
       'fcs_coaches_poll_y', 'away_comm

In [41]:
both_ranked_df = gamesRankings_df[(gamesRankings_df['away_ap_rank'] > 0) & (gamesRankings_df['home_ap_rank'] > 0)]

In [ ]:
both_ranked_df = both_ranked_df[['id', 'season', 'week', 'seasonType', 'startDate',
     'homeId', 'homeTeam', 'homePoints', 'home_ap_rank', 'home_coaches_rank', 'home_committee_rank', 'homePregameElo',
    'awayId', 'awayTeam', 'awayPoints', 'away_ap_rank', 'away_coaches_rank', 'away_committee_rank', 'awayPregameElo',
     ]]

both_ranked_df

,id,season,week,seasonType,startDate,homeId,homeTeam,homePoints,home_ap_rank,home_coaches_rank,home_committee_rank,homePregameElo,awayId,awayTeam,awayPoints,away_ap_rank,away_coaches_rank,away_committee_rank,awayPregameElo
4,63072,2000,1,regular,2000-08-27T00:00:00.000Z,30,USC,29.0,15.0,NaN,NaN,1641.0,213,Penn State,5.0,22.0,NaN,NaN,1789.0
28,63123,2000,2,regular,2000-09-02T00:00:00.000Z,2633,Tennessee,19.0,13.0,NaN,NaN,1834.0,2572,Southern Miss,16.0,22.0,NaN,NaN,1685.0
76,63158,2000,3,regular,2000-09-09T00:00:00.000Z,87,Notre Dame,24.0,23.0,NaN,NaN,1639.0,158,Nebraska,27.0,1.0,NaN,NaN,1963.0
108,63185,2000,3,regular,2000-09-09T00:00:00.000Z,264,Washington,34.0,15.0,NaN,NaN,1640.0,2390,Miami,29.0,4.0,NaN,NaN,1801.0
132,63234,2000,4,regular,2000-09-16T00:00:00.000Z,26,UCLA,23.0,14.0,NaN,NaN,1495.0,130,Michigan,20.0,3.0,NaN,NaN,1846.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44984,401756945,2025,11,regular,2025-11-08T17:00:00.000Z,2641,Texas Tech,29.0,9.0,9.0,8.0,1903.0,252,BYU,7.0,8.0,8.0,7.0,1722.0
45169,401752763,2025,11,regular,2025-11-08T20:30:00.000Z,142,Missouri,17.0,19.0,17.0,22.0,1779.0,245,Texas A&M,38.0,3.0,3.0,3.0,1859.0
45252,401754597,2025,12,regular,2025-11-15T17:00:00.000Z,221,Pittsburgh,NaN,23.0,23.0,22.0,1668.0,87,Notre Dame,NaN,9.0,9.0,9.0,2186.0
45479,401752765,2025,12,regular,2025-11-15T20:30:00.000Z,333,Alabama,NaN,4.0,4.0,4.0,1977.0,201,Oklahoma,NaN,11.0,10.0,11.0,1794.0


In [82]:
results = []

In [83]:
ap_both_res, ap_both_ranked_df = calcRankWinPercent(
    gamesRankings_df, 'away_ap_rank', 'home_ap_rank', True)

results.append(ap_both_res)

pd.DataFrame(results)


(1462, 51)
964/1462->0.6594


,Poll,Scope,Wins,Total,Accuracy
0,ap_rank,both ranked,964,1462,0.659371


In [84]:
ap_either_res, ap_either_ranked_df = calcRankWinPercent(
    gamesRankings_df, 'away_ap_rank', 'home_ap_rank', False)

results.append(ap_either_res)

pd.DataFrame(results)

(7334, 51)
5792/7334->0.7897


,Poll,Scope,Wins,Total,Accuracy
0,ap_rank,both ranked,964,1462,0.659371
1,ap_rank,either ranked,5792,7334,0.789746


In [86]:
coaches_both_res, coaches_both_ranked_df = calcRankWinPercent(
    gamesRankings_df, 'away_coaches_rank', 'home_coaches_rank', True)

results.append(coaches_both_res)

pd.DataFrame(results)


(1422, 51)
937/1422->0.6589


,Poll,Scope,Wins,Total,Accuracy
0,ap_rank,both ranked,964,1462,0.659371
1,ap_rank,either ranked,5792,7334,0.789746
2,coaches_rank,both ranked,937,1422,0.658931


In [87]:
coaches_either_res, coaches_either_ranked_df = calcRankWinPercent(
    gamesRankings_df, 'away_coaches_rank', 'home_coaches_rank', False)

results.append(coaches_either_res)

pd.DataFrame(results)

(7079, 51)
5581/7079->0.7884


,Poll,Scope,Wins,Total,Accuracy
0,ap_rank,both ranked,964,1462,0.659371
1,ap_rank,either ranked,5792,7334,0.789746
2,coaches_rank,both ranked,937,1422,0.658931
3,coaches_rank,either ranked,5581,7079,0.788388


In [88]:
cmt_both_res, cmt_both_ranked_df = calcRankWinPercent(
    gamesRankings_df, 'away_committee_rank', 'home_committee_rank', True)

results.append(cmt_both_res)

pd.DataFrame(results)

(251, 51)
162/251->0.6454


,Poll,Scope,Wins,Total,Accuracy
0,ap_rank,both ranked,964,1462,0.659371
1,ap_rank,either ranked,5792,7334,0.789746
2,coaches_rank,both ranked,937,1422,0.658931
3,coaches_rank,either ranked,5581,7079,0.788388
4,committee_rank,both ranked,162,251,0.645418


In [89]:
cmt_either_res, cmt_either_ranked_df = calcRankWinPercent(
    gamesRankings_df, 'away_committee_rank', 'home_committee_rank', False)

results.append(cmt_either_res)

pd.DataFrame(results)

(1071, 51)
811/1071->0.7572


,Poll,Scope,Wins,Total,Accuracy
0,ap_rank,both ranked,964,1462,0.659371
1,ap_rank,either ranked,5792,7334,0.789746
2,coaches_rank,both ranked,937,1422,0.658931
3,coaches_rank,either ranked,5581,7079,0.788388
4,committee_rank,both ranked,162,251,0.645418
5,committee_rank,either ranked,811,1071,0.757236
